# Visualization Using TensorBoard
Computation graphs are hard to imagine. The progress of training is also hard to imagine. Let's visualize them. Fortunately, TensorFlow makes this very very easy with TensorBoard. 

In [9]:
import tensorflow as tf

In the section 4, we had the following code:

In [10]:
a = tf.add(2,3)       

with tf.Session() as sess:
    print('a:',sess.run(a))

a: 5


Let's rewrite this code using more TensorFlow:

In [11]:
a = tf.constant(2, name = 'a')
b = tf.constant(3, name = 'b')
sum_ = tf.add(a,b, name = 'sum_')       

with tf.Session() as sess:
    print('sum_:',sess.run(sum_))

sum_: 5


## Basic Visualization

How does the computation graph of this code look like? Let's visualize it using TensorBoard. To do this, you just need to add one line to the code:

In [12]:
tf.reset_default_graph() # Clearing all tensors before this
a = tf.constant(2, name = 'a')
b = tf.constant(3, name = 'b')
sum_ = tf.add(a,b, name = 'sum_')       

with tf.Session() as sess:
    print('sum_:',sess.run(sum_))
    writer = tf.summary.FileWriter('log/simple', sess.graph) # NEW!
    writer.close()

sum_: 5


This should create a new directory called `log` to where this Jupyter Notebook is located. Now, run the following on your local terminal, 

`tensorboard --logdir=log/simple`

Then, head to `http://localhost:6006` and see for yourself that the graph is being visualized!

## Logistic Regression Visualization

### Simple Visualization

In [13]:
tf.reset_default_graph() # Clearing all tensors before this

In [14]:


# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# Variables
M = 784
K = 10
X = tf.placeholder(tf.float32, shape=[None, M])
Y = tf.placeholder(tf.float32, shape=[None, K])
W = tf.Variable(tf.zeros([M, K]))
b = tf.Variable(tf.zeros([K]))

# Model
Yhat = tf.nn.softmax(tf.matmul(X, W) + b)

# Loss
loss = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(Yhat), axis=1))

# Optimizer
learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [15]:
# Train
num_epochs  = 25
batch_size  = 100

with tf.Session() as sess:
    writer = tf.summary.FileWriter('log/logistic1', sess.graph) # NEW!
    writer.close()

### Better: Name the variables

In [16]:
tf.reset_default_graph() # Clearing all tensors before this

In [17]:


# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# Variables
M = 784
K = 10
X = tf.placeholder(tf.float32, shape=[None, M], name='Input-Images')
Y = tf.placeholder(tf.float32, shape=[None, K], name='Output-Labels')
W = tf.get_variable('Weights', initializer=tf.zeros([M, K]))
b = tf.get_variable('Bias', initializer=tf.zeros([K]))

# Model
Yhat = tf.nn.softmax(tf.matmul(X, W) + b, name='Prediction-Labels')

# Loss
loss = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(Yhat), axis=1), name="Cross-Entropy-Loss")

# Optimizer
learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate, name="SGD-Optimizer")
update    = optimizer.minimize(loss)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [18]:
# Train
num_epochs  = 25
batch_size  = 100

with tf.Session() as sess:
    writer = tf.summary.FileWriter('log/logistic2', sess.graph) # NEW!
    writer.close()

### Even Better: Group Variables Together

In [19]:
tf.reset_default_graph() # Clearing all tensors before this

In [20]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

M = 784
K = 10
with tf.name_scope('data'):
    X = tf.placeholder(tf.float32, shape=[None, M], name='Input-Images')
    Y = tf.placeholder(tf.float32, shape=[None, K], name='Output-Labels')
    W = tf.get_variable('Weights', initializer=tf.zeros([M, K]))
    b = tf.get_variable('Bias', initializer=tf.zeros([K]))

with tf.name_scope('model'):
    Yhat = tf.nn.softmax(tf.matmul(X, W) + b, name='Prediction-Labels')

with tf.name_scope('loss'):
    loss = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(Yhat), axis=1), name="Cross-Entropy-Loss")

with tf.name_scope('optimizer'):
    learning_rate = 0.01
    optimizer = tf.train.GradientDescentOptimizer(learning_rate, name="SGD-Optimizer")
    update    = optimizer.minimize(loss)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [21]:
# Train
num_epochs  = 25
batch_size  = 100

with tf.Session() as sess:
    writer = tf.summary.FileWriter('log/logistic3', sess.graph) # NEW!
    writer.close()

### Even Even Better: Create Summaries to Track Training

In [22]:
tf.reset_default_graph() # Clearing all tensors before this

In [23]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

M = 784
K = 10
with tf.name_scope('data'):
    X = tf.placeholder(tf.float32, shape=[None, M], name='Input-Images')
    Y = tf.placeholder(tf.float32, shape=[None, K], name='Output-Labels')
    W = tf.get_variable('Weights', initializer=tf.zeros([M, K]))
    b = tf.get_variable('Bias', initializer=tf.zeros([K]))

with tf.name_scope('model'):
    Yhat = tf.nn.softmax(tf.matmul(X, W) + b, name='Prediction-Labels')

with tf.name_scope('loss'):
    loss = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(Yhat), axis=1), name="Cross-Entropy-Loss")

with tf.name_scope('optimizer'):
    learning_rate = 0.01
    optimizer = tf.train.GradientDescentOptimizer(learning_rate, name="SGD-Optimizer")
    update    = optimizer.minimize(loss)

with tf.name_scope('summaries'):
    tf.summary.scalar('loss', loss)
    tf.summary.histogram('histogram-loss', loss)
    summary_op = tf.summary.merge_all()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [30]:
# Train
num_epochs  = 25
batch_size  = 100

with tf.Session() as sess:
    writer = tf.summary.FileWriter('log/logistic4', sess.graph)
    
    sess.run(tf.global_variables_initializer())
    total_batch = int(mnist.train.num_examples/batch_size)
    for epoch in range(num_epochs):
        average_cost = 0
        for batch in range(total_batch):
            batch_X, batch_Y = mnist.train.next_batch(batch_size)
            _, c = sess.run([update, loss], feed_dict={X: batch_X,
                                                       Y: batch_Y})
            average_cost += c / total_batch
            summary = sess.run(summary_op, feed_dict={X: batch_X,
                                                    Y: batch_Y})
            global_step = epoch*total_batch + batch
            writer.add_summary(summary, global_step=global_step)
        print("Epoch:",epoch,"Cost:",average_cost)
        
        
        
    writer.close()

Epoch: 0 Cost: 1.1840218196131986
Epoch: 1 Cost: 0.6652586256374015
Epoch: 2 Cost: 0.5528820904276592
Epoch: 3 Cost: 0.4987890945781363
Epoch: 4 Cost: 0.4654299976067115
Epoch: 5 Cost: 0.4423869885639709
Epoch: 6 Cost: 0.42576095984740686
Epoch: 7 Cost: 0.4121414015780799
Epoch: 8 Cost: 0.40144512360746204
Epoch: 9 Cost: 0.3926681646975608
Epoch: 10 Cost: 0.3847223829410292
Epoch: 11 Cost: 0.37781452579931785
Epoch: 12 Cost: 0.3724527456814593
Epoch: 13 Cost: 0.3674735837362031
Epoch: 14 Cost: 0.3625684221766213
Epoch: 15 Cost: 0.35839478755539117
Epoch: 16 Cost: 0.3549837627194145
Epoch: 17 Cost: 0.35159457144412093
Epoch: 18 Cost: 0.3479967925765299
Epoch: 19 Cost: 0.3457087926972995
Epoch: 20 Cost: 0.3428178538517521
Epoch: 21 Cost: 0.34027049026706
Epoch: 22 Cost: 0.3378532365235416
Epoch: 23 Cost: 0.3361319926652042
Epoch: 24 Cost: 0.333278747173873


## Exercise
Nicely visualize the linear regression you implemented in the previous exercise.
